In [26]:
from typing import List, Optional, Tuple, Union
import numpy as np
import particle_simulators as ps
from importlib import reload
import visualize as vis
reload(vis)
reload(ps)

<module 'particle_simulators' from '/Users/masomatics/Library/CloudStorage/GoogleDrive-koyama.masanori@gmail.com/マイドライブ/mptf_exps/particle_simulators.py'>

In [27]:
anchor1= np.array([1., 0, 0]) 
anchor2= np.array([0, 1., 0])

anchors = np.vstack((anchor1, anchor2))
print(anchors)

[[1. 0. 0.]
 [0. 1. 0.]]


In [28]:
import numpy as np
from typing import Tuple

def cart2sph(xyz: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Cartesian → spherical:  returns (theta, phi) in radians.
    theta ∈ [0, π]   polar angle from +z
    phi   ∈ [0, 2π)  azimuth from +x toward +y
    """
    x, y, z = xyz[..., 0], xyz[..., 1], xyz[..., 2]
    theta = np.arccos(np.clip(z, -1.0, 1.0))        # avoid NaNs
    phi   = np.arctan2(y, x) % (2.0 * np.pi)
    return theta, phi

def sph2cart(theta: np.ndarray, phi: np.ndarray) -> np.ndarray:
    """Spherical → Cartesian (unit radius)."""
    sinT = np.sin(theta)
    return np.stack([sinT * np.cos(phi),
                     sinT * np.sin(phi),
                     np.cos(theta)], axis=-1)

def constant_speed_sphere_path(
    anchors: np.ndarray,          # (M, 3)   starting points (unit)
    *,
    theta_dot: float,             # dθ/dt  [rad / time-unit]
    eta_dot: float,               # dφ/dt  [rad / time-unit]
    T: float = 15.0,
    dt: float = 0.1,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Return a trajectory where every anchor moves with constant angular
    velocity (theta_dot, eta_dot) on the unit sphere.

    Returns
    -------
    traj : ndarray, shape (num_steps, M, 3)
        Cartesian positions for every anchor at every step.
    t_grid : ndarray, shape (num_steps,)
        Time stamps.
    """
    anchors = np.asarray(anchors, dtype=float)
    assert anchors.ndim == 2 and anchors.shape[1] == 3, "anchors must be (M,3)"
    # normalise defensively
    anchors /= np.linalg.norm(anchors, axis=1, keepdims=True)

    M = anchors.shape[0]
    theta0, phi0 = cart2sph(anchors)               # (M,)

    num_steps = int(T / dt) + 1
    t_grid = np.linspace(0.0, T, num_steps)

    # evolve angles linearly
    theta = theta0[:, None] + theta_dot * t_grid   # (M, num_steps)
    phi   = phi0[:, None]   + eta_dot   * t_grid

    # keep angles in canonical ranges
    theta = np.mod(theta, 2.0 * np.pi)
    phi   = np.mod(phi,   2.0 * np.pi)

    # convert back to Cartesian for every time step
    traj = sph2cart(theta[..., None], phi[..., None])   # (M, num_steps, 3)
    traj = np.swapaxes(traj, 0, 1)                      # → (num_steps, M, 3)
    return traj, t_grid


# ---------------------------------------------------------------------
# Example usage
# ---------------------------------------------------------------------
if __name__ == "__main__":
    anchor1 = np.array([1.0, 0.0, 0.0])
    anchor2 = np.array([0.0, 1.0, 0.0])
    anchors = np.vstack((anchor1, anchor2))

    theta_dot = 0.1       # rad / unit-time
    eta_dot   = 0.25
    traj, t = constant_speed_sphere_path(
        anchors,
        theta_dot=theta_dot,
        eta_dot=eta_dot,
        T=15.0,
        dt=0.1,
    )

    print("trajectory shape:", traj.shape)   # (num_steps, 2, 3)
    print("first frame:", traj[0])
    print("last  frame:", traj[-1])


trajectory shape: (151, 2, 1, 3)
first frame: [[[1.000000e+00 0.000000e+00 6.123234e-17]]

 [[6.123234e-17 1.000000e+00 6.123234e-17]]]
last  frame: [[[-0.05804407 -0.04043065 -0.99749499]]

 [[ 0.04043065 -0.05804407 -0.99749499]]]


In [29]:
anchor_traj = traj.squeeze(2)
anchor_traj.shape

(151, 2, 3)

In [30]:
myseed= 10
weight = 2
T = 15
reload(ps)
reload(vis)

m_sim = ps.MobileAnchorSimulator(anchor_weight=weight, anchor_time_series=anchor_traj)

z_list, t_grid = m_sim.simulate()
for k in range(len(z_list)):
    print(len(z_list[k]))

66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66


In [31]:
z_list[0].shape

(66, 3)

In [21]:


vis.render(z_list, 3,  t_grid, m=64, rootdrivepath='./figs', movie=True, fps=10,  title=f"mobile_anchors_weight{weight}", interpolate=True) 

render: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151/151 [00:08<00:00, 18.26it/s]


GIF saved → ./figs/sphere/15-22-46_mobile_anchors_weight2_movie.gif


66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
